# SQL 

## Цель исследования:

Анализ базы данных сервиса для чтения книг

**Задачи:**
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — с исключением брошюр;
* Определить автора с самой высокой средней оценкой книг (учесть только книги с 50 и более оценками);
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок;
* Сделать выводы по каждой задаче.


## Инструменты
### Библиотеки

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
from IPython.display import Markdown, display, display_html

### Подключение к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': '--', # Интеллектуальная собственность Яндекс.Практикума 
'host': '--', # Интеллектуальная собственность Яндекс.Практикума
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas

con = engine.connect()

### Функции

In [3]:
def sql(query, prnt=True):
    data = pd.io.sql.read_sql(sql=text(query), con = con)
    if prnt==True:
        display(data)
        return
    return data

In [4]:
def read_table(tables):
    for i in tables:
        print() # Отступ
        display(Markdown(f'<b><big>{i}</big></b>'))
        sql('''
            SELECT * 
            FROM ''' + i +
           ''' LIMIT 5 ''')
        t_shape = sql('''
            SELECT count(*) 
            FROM ''' + i, 
                      False # Не отображаем таблицу по умолчанию
                     ).at[0,'count']
        display(Markdown(f'Количество строк в таблице: **{t_shape}**'))
    return

## Чтение и изучение данных

In [5]:
read_table(['books', 'authors', 'publishers', 'ratings', 'reviews'])

<b><big>books</big></b>

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество строк в таблице: **1000**

<b><big>authors</big></b>

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


...

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество строк в таблице: **6456**

<b><big>reviews</big></b>

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество строк в таблице: **2793**

### Описание данных
Таблица **`books`** 
Содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица **`authors`** 
Содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица **`publishers`**
Содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

Таблица **`ratings`**
Содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица **`reviews`**
Содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

## Анализ данных

### Количество вышедших книг после 1 января 2000 года

In [6]:
sql('''

SELECT DISTINCT count(book_id) AS book_cnt
FROM books
WHERE publication_date >= '2000-01-01' 
LIMIT 5

''')

,book_cnt
0,821


После 1 января 2000 (включительно) года вышла 821 книга.

### Количество обзоров и средняя оценка для каждой книги

In [7]:
sql('''

SELECT b.title,
       avg(rat.rating) AS avg_rating,
       count(rev.review_id) AS cnt_review
FROM books AS b
LEFT JOIN ratings AS rat
    ON b.book_id = rat.book_id
LEFT JOIN reviews AS rev
    ON b.book_id = rev.book_id
GROUP BY  b.title
ORDER BY  avg_rating desc, cnt_review desc

''')

,title,avg_rating,cnt_review
0,A Dirty Job (Grim Reaper #1),5.00,16
1,School's Out—Forever (Maximum Ride #2),5.00,12
2,Moneyball: The Art of Winning an Unfair Game,5.00,9
3,In the Hand of the Goddess (Song of the Liones...,5.00,6
4,The Walking Dead Book One (The Walking Dead #...,5.00,4
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,2.25,12
995,Drowning Ruth,2.00,9
996,Junky,2.00,4
997,His Excellency: George Washington,2.00,4


Книга с максимальным средним рейтингом и максимальным количеством обзоров **"A Dirty Job (Grim Reaper #1)"**

### Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключая из анализа брошюры)

In [8]:
sql('''

SELECT publisher,
       count(b.book_id) AS count_books
FROM books AS b
INNER JOIN publishers AS p
    ON b.publisher_id = p.publisher_id
WHERE b.num_pages >50
GROUP BY  p.publisher
ORDER BY  count_books DESC limit 1

''')

,publisher,count_books
0,Penguin Books,42


Наибольшее количество книг (исключая брошюры) выпустило издательство **"Penguin Books"**. Оно выпустило 42 книги.

### Автор с самой высокой средней оценкой книг (учитывая самые оцениваемые книги- книги с 50 и более оценками)

In [9]:
# Запрос с использованием временной таблицы
sql('''
with cnt_rat AS 
        (SELECT b.book_id,
                count(rat.rating_id)
        FROM books AS b
        INNER JOIN ratings AS rat
            ON b.book_id = rat.book_id
        GROUP BY  b.book_id
        HAVING count(rat.rating_id) >=50
        ORDER BY  avg(rat.rating) DESC 
        )
SELECT a.author,
       avg(rat.rating) AS avg_rating
FROM cnt_rat AS cnt
LEFT JOIN books AS b
    ON cnt.book_id = b.book_id
LEFT JOIN ratings AS rat
    ON b.book_id = rat.book_id
INNER JOIN authors AS a
    ON b.author_id = a.author_id
GROUP BY  a.author
ORDER BY  avg(rat.rating) DESC 
LIMIT 1
''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


In [10]:
# Альтернативный запрос с использованием оконной функции 
sql('''
SELECT a.author, avg(rat.rating) as avg_rating
FROM books b
INNER JOIN 
        (SELECT book_id, 
                rating, 
                count(rating) over(partition by book_id) as cnt_of_rating
         FROM ratings
         ) rat
      ON b.book_id = rat.book_id 
      AND cnt_of_rating >=50
LEFT JOIN authors AS a
    ON b.author_id = a.author_id 
group by a.author
ORDER BY avg_rating DESC
LIMIT 1
''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг с более 50 оценок за книгу - **J.K. Rowling** с книгами иллюстрации **Mary GrandPré** 

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
sql('''
with cnt_rat AS 
    (SELECT username,
    count(rating_id) AS rating_cnt
    FROM ratings
    GROUP BY  username
    HAVING count(rating_id) > 48 
    ), 
cnt_rev AS 
    (SELECT username,
     count(review_id) AS review_cnt
    FROM reviews
    GROUP BY  username 
    )
SELECT avg(rev.review_cnt) AS avg_review
FROM cnt_rat AS rat
LEFT JOIN cnt_rev AS rev
    ON rat.username = rev.username
''')

,avg_review
0,24.0


Пользователи, поставившие более 48 оценок книгам в среднем оставили по **24** обзора.

### Выводы
Из тысячи книг в базе данных **821** книга (включая брошюры) вышла после 1 января 2020 года.

За это время самым продуктивным изданием было издание **Penguin Books**.

А автором с самой большой среней оценкой за книги, учитывая только наиболее оцениваемые (более 50 оценок) **J.K. Rowling** с серией книг о Гарри Поттере с иллюстрациями **Mary GrandPré** 